In [24]:
import json
import datetime
import cPickle as pickle

import numpy as np

from astropy.time import Time

# Accessing twitter/downloading tweets 

In [ ]:
import tweepy

In [ ]:
def search_all(query, api, maxcount):
    """
    Get up to `maxcount` tweets from the search query `query`, using the `api` object.
    """
    res = []
    lastid=None
    while len(res) < maxcount:
        if lastid is None:
            s = api.search(query, count=100)
        else:
            s = api.search(query, count=100, max_id=lastid-1)
        if len(s)<1:
            print('finished search')
            break
        res.extend(s)
        lastid = s[-1].id
        print('with',len(res),'we go back to',s[-1].created_at)
    return res
def write_tweets(tws, fname):
    """
    Write out the tweets to JSON and pickle files
    """
    with open(fname+'.json','w') as f:
        json.dump([si._json for si in tws], f)
    with open(fname+'.pickle','w') as f:
        pickle.dump(tws, f, -1)

You'll need to creat a twitter app, generate an access token, and then set the `consumer_key`, `consumer_secret`, `acess_token`, and `access_token_secret` variables.

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(acess_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
tweets = search_all('#LGAstat',api,10000)

In [ ]:
dates = [e.created_at for e in tweets]
jds = Time(dates).jd

In [ ]:
#this cell writes them out to local files
write_tweets(tweets, 'tweets')
np.save('tweets_jd', jds)

# Or load from local file 

In [ ]:
import tweepy

In [ ]:
with open('tweetstothnight.pickle') as f:
    tweets = pickle.load(f)

In [4]:
jds = np.load('tweets_jd_tothnight.npy')

# Plots 

In [2]:
%matplotlib inline
import mpld3
from matplotlib import pyplot as plt

In [27]:
midnight528jd = Time(datetime.datetime(2015, 5, 28, 0)).jd

In [30]:
plt.step(np.sort(jds-midnight528jd), np.arange(len(jds)))
plt.xlabel('jd - LGAjd')
plt.ylabel('N')
mpld3.save_html(plt.gcf(), 'tweetjds.html')
mpld3.display()